In [ ]:
#https://github.com/akshaybahadur21/Drowsiness_Detection/blob/master/drownsiness_detection.ipynb

In [1]:
from scipy.spatial import distance
'''calcuates distance by different ways'''

'calcuates distance by different ways'

In [2]:
import playsound


In [3]:
from imutils import face_utils

''' The mouth can be accessed through points [48, 68].
The right eyebrow through points [17, 22].
The left eyebrow through points [22, 27].
The right eye using [36, 42].
The left eye with [42, 48].
The nose using [27, 35].
And the jaw via [0, 17].
These mappings are encoded inside the FACIAL_LANDMARKS_IDXS  dictionary inside face_utils of the imutils library:'''

' The mouth can be accessed through points [48, 68].\nThe right eyebrow through points [17, 22].\nThe left eyebrow through points [22, 27].\nThe right eye using [36, 42].\nThe left eye with [42, 48].\nThe nose using [27, 35].\nAnd the jaw via [0, 17].\nThese mappings are encoded inside the FACIAL_LANDMARKS_IDXS  dictionary inside face_utils of the imutils library:'

In [4]:
import imutils
'''IMUTILS =This package includes a series of OpenCV + 
convenience functions that perform basics tasks 
such as translation, rotation, resizing, and skeletonization.'''



'IMUTILS =This package includes a series of OpenCV + \nconvenience functions that perform basics tasks \nsuch as translation, rotation, resizing, and skeletonization.'

In [5]:
import dlib

'''The facial landmark detector implemented inside dlib produces 68 
(x, y)-coordinates that map to specific facial structures.
These 68 point mappings were obtained by training a shape predictor on the labeled iBUG 300-W dataset
https://www.pyimagesearch.com/2017/04/10/detect-eyes-nose-lips-jaw-dlib-opencv-python/'''

'The facial landmark detector implemented inside dlib produces 68 \n(x, y)-coordinates that map to specific facial structures.\nThese 68 point mappings were obtained by training a shape predictor on the labeled iBUG 300-W dataset\nhttps://www.pyimagesearch.com/2017/04/10/detect-eyes-nose-lips-jaw-dlib-opencv-python/'

In [6]:
import cv2

In [8]:
def eye_aspect_ratio(eye):
    A=distance.euclidean(eye[1],eye[5])
    B=distance.euclidean(eye[2],eye[4])
    C = distance.euclidean(eye[0],eye[3])
    ear= (A + B) / (2.0 * C)
    return ear


In [17]:


thresh = 0.25
frame_check=20
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
type(detect)
#https://www.youtube.com/watch?v=4ESLTAd3IOM==HOG


dlib.fhog_object_detector

In [10]:
# grab the indexes of the facial landmarks for the left and
# right eye, respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [25]:
#0== default camera
#1 or -1== any connected camera
cap=cv2.VideoCapture(0)
#monitor the ear
flag=0
while True:
    #read image
    ret, frame=cap.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #applies dlib’s face detector to find and locate the face(s) in the image.
    subjects = detect(gray, 0)
    # loop over the face detections
    for subject in subjects:
        # determine the facial landmarks for the face region, then
		# convert the facial landmark (x, y)-coordinates to a NumPy
		# array
        shape = predict(gray, subject)
        shape = face_utils.shape_to_np(shape)#converting to NumPy Array

        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0
        #https://www.learnopencv.com/convex-hull-using-opencv-in-python-and-c/
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        if ear < thresh:
            flag += 1
            #print (flag)
            if flag >= frame_check:
                sound_alarm('alarm.wav')
                cv2.putText(frame, "**************WAKE UP!****************", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "**************WAKE UP!****************", (10,325),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            flag = 0
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        cv2.destroyAllWindows()
        cap.release()
        break